In [47]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
os.chdir(r'd:\Shraddha\Automatic Number Plate Recognition, OCR Web App in Python\[TutsNode.com] - Automatic Number Plate Recognition, OCR Web App in Python\Introduction\Project_Files\Labeling')


## Read the data created by reading XML files from images folder

In [48]:
df = pd.read_csv('labels.csv')
df.head()

,filepath,xmin,xmax,ymin,ymax
0,images\N1.xml,1093,1396,645,727
1,images\N100.xml,134,301,312,350
2,images\N101.xml,31,139,128,161
3,images\N102.xml,164,316,216,243
4,images\N103.xml,813,1067,665,724


In [49]:
import xml.etree.ElementTree as xet

In [50]:
filename = df['filepath'][0]

## Function to get path of xml files within folder

In [51]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('.\images',filename_image)
    return filepath_image

In [52]:
print(os.getcwd())
getFilename(filename)

d:\Shraddha\Automatic Number Plate Recognition, OCR Web App in Python\[TutsNode.com] - Automatic Number Plate Recognition, OCR Web App in Python\Introduction\Project_Files\Labeling


'.\\images\\N1.jpeg'

In [53]:
image_path = list(df['filepath'].apply(getFilename))
image_path

['.\\images\\N1.jpeg',
 '.\\images\\N100.jpeg',
 '.\\images\\N101.jpeg',
 '.\\images\\N102.jpeg',
 '.\\images\\N103.jpeg',
 '.\\images\\N104.jpeg',
 '.\\images\\N105.jpeg',
 '.\\images\\N106.jpeg',
 '.\\images\\N107.jpeg',
 '.\\images\\N108.jpeg',
 '.\\images\\N109.jpeg',
 '.\\images\\N11.jpeg',
 '.\\images\\N110.jpeg',
 '.\\images\\N111.jpeg',
 '.\\images\\N112.jpeg',
 '.\\images\\N113.jpeg',
 '.\\images\\N114.jpeg',
 '.\\images\\N116.jpeg',
 '.\\images\\N117.jpeg',
 '.\\images\\N118.jpeg',
 '.\\images\\N119.jpeg',
 '.\\images\\N12.jpeg',
 '.\\images\\N120.jpeg',
 '.\\images\\N121.jpeg',
 '.\\images\\N122.jpeg',
 '.\\images\\N123.jpeg',
 '.\\images\\N124.jpeg',
 '.\\images\\N126.jpeg',
 '.\\images\\N127.jpeg',
 '.\\images\\N128.jpeg',
 '.\\images\\N129.jpeg',
 '.\\images\\N130.jpeg',
 '.\\images\\N131.jpeg',
 '.\\images\\N132.jpeg',
 '.\\images\\N133.jpeg',
 '.\\images\\N134.jpeg',
 '.\\images\\N135.jpeg',
 '.\\images\\N136.jpeg',
 '.\\images\\N137.jpeg',
 '.\\images\\N138.jpeg',
 '.\

# Verfiy image and output

In [31]:
# file_path = image_path[0]
# file_path

In [32]:
# img = cv2.imread(r"D:\Shraddha\Automatic Number Plate Recognition, OCR Web App in Python\[TutsNode.com] - Automatic Number Plate Recognition, OCR Web App in Python\Introduction\Project_Files\1_Labeling\images\N1.jpeg")
# cv2.namedWindow('example',cv2.WINDOW_NORMAL)
# cv2.imshow('example',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [33]:
# cv2.rectangle(img,(1093,645),(1396,727),(0,255,0),3)
# cv2.namedWindow('example',cv2.WINDOW_NORMAL)
# cv2.imshow('example',img)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

# Data Preprocessing

In [54]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img,img_to_array

In [55]:
labels = df.iloc[:,1:].values

In [56]:
data = []
output = []

for ind in range(len(image_path)):
    image = image_path[0]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape
    # preprocessing
    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_Arr = load_image_arr/255.0 #normalization
    # normalization to labels
    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w, xmax/w
    nymin,nymax = ymin/h, ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) #normalizaed output
    # ------------------ append
    data.append(norm_load_image_Arr)
    output.append(label_norm)

In [57]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [58]:
X.shape , y.shape

((225, 224, 224, 3), (225, 4))

## Splitting data into train and test data

In [59]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((180, 224, 224, 3), (45, 224, 224, 3), (180, 4), (45, 4))

# Deep Learning Model

In [60]:
from tensorflow.keras.applications import MobileNetV2, InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout , Flatten , Input
from tensorflow.keras.models import Model
import tensorflow as tf

In [61]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False,input_tensor=Input(shape=(224,224,3)))
# TRUE if you want wait to change
inception_resnet.trainable = False

## Build model

In [62]:
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
# write if you want any extra neuron
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)
# ---------------- model
# input for model is inception_resnet and output is headmodel
model = Model(inputs=inception_resnet.input,outputs=headmodel)

## Compile Model

In [63]:
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_406 (Conv2D)             (None, 111, 111, 32) 864         input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_406 (BatchN (None, 111, 111, 32) 96          conv2d_406[0][0]                 
__________________________________________________________________________________________________
activation_406 (Activation)     (None, 111, 111, 32) 0           batch_normalization_406[0][0]    
____________________________________________________________________________________________

# Model Trainning (Object detection model)

In [64]:
from tensorflow.keras.callbacks import TensorBoard

In [67]:
tfb = TensorBoard(r'D:\Shraddha\Automatic Number Plate Recognition, OCR Web App in Python\[TutsNode.com] - Automatic Number Plate Recognition, OCR Web App in Python\Introduction\Project_Files\Labeling\object_detection')

In [76]:
# history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=100,validation_data=(x_test,y_test),callbacks=[tfb])
history = model.fit(x=x_train,y=y_train,epochs=100)

Epoch 1/100
 2/18 [==>...........................] - ETA: 23s - loss: 0.1912

NotFoundError: Failed to create a NewWriteableFile: D:\Shraddha\Automatic Number Plate Recognition, OCR Web App in Python\[TutsNode.com] - Automatic Number Plate Recognition, OCR Web App in Python\Introduction\Project_Files\Labeling\object_detection\train\plugins\profile\2021_10_30_17_46_11\SHRADDHA.trace.json.gz : The system cannot find the path specified.
; No such process

In [73]:
hist_df = pd.DataFrame(history.history) 

In [74]:
hist_json_file = 'D:\Shraddha\Automatic Number Plate Recognition, OCR Web App in Python\[TutsNode.com] - Automatic Number Plate Recognition, OCR Web App in Python\Introduction\Project_Files\Labeling\object_detection\history.json' 
with open(hist_json_file, mode='w') as f:
    hist_df.to_json(f)

In [75]:
hist_csv_file = 'D:\Shraddha\Automatic Number Plate Recognition, OCR Web App in Python\[TutsNode.com] - Automatic Number Plate Recognition, OCR Web App in Python\Introduction\Project_Files\Labeling\object_detection\history.csv' 

# hist_csv_file = 'history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)